# Notebook 07 — Rolling Window Analysis

## Time-Varying OU Parameters (24-Month Rolling Window)

This notebook tracks how the OU model parameters evolve over time by re-estimating them on a rolling 24-month window.

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy import stats as sp_stats
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.figsize': (12, 6), 'figure.dpi': 150, 'savefig.dpi': 300,
    'font.size': 11, 'axes.titlesize': 14, 'axes.labelsize': 12,
    'legend.fontsize': 10, 'font.family': 'serif'
})
print('Libraries loaded.')

In [ ]:
# ─── Load Data ───────────────────────────────────────────────────────────────
usd = pd.read_csv('../data/processed/spreads_usd_new_amount.csv', parse_dates=['date'], index_col='date')
khr = pd.read_csv('../data/processed/spreads_khr_new_amount.csv', parse_dates=['date'], index_col='date')
S_usd = usd['spread'].values
S_khr = khr['spread'].values
dates = usd.index
dt = 1/12
window = 24  # 24-month rolling window

# OU estimation
def ou_neg_log_likelihood(params, data, dt):
    kappa, theta, sigma = params
    if kappa <= 0 or sigma <= 0:
        return 1e10
    n = len(data) - 1
    exp_kdt = np.exp(-kappa * dt)
    m = theta + (data[:-1] - theta) * exp_kdt
    v = (sigma**2 / (2 * kappa)) * (1 - np.exp(-2 * kappa * dt))
    if v <= 0:
        return 1e10
    residuals = data[1:] - m
    ll = -0.5 * n * np.log(2 * np.pi) - 0.5 * n * np.log(v) - 0.5 * np.sum(residuals**2) / v
    return -ll

def estimate_ou(data, dt):
    best_nll = np.inf
    best_x = None
    for k0 in [0.5, 1.0, 2.0, 5.0, 10.0, 20.0]:
        result = minimize(ou_neg_log_likelihood,
                         [k0, np.mean(data), np.std(np.diff(data))*np.sqrt(12)],
                         args=(data, dt), method='Nelder-Mead',
                         options={'maxiter': 50000, 'xatol': 1e-10, 'fatol': 1e-10})
        if result.fun < best_nll and result.x[0] > 0 and result.x[2] > 0:
            best_nll = result.fun
            best_x = result.x
    if best_x is None:
        return {'kappa': np.nan, 'theta': np.nan, 'sigma': np.nan}
    return {'kappa': best_x[0], 'theta': best_x[1], 'sigma': best_x[2]}

print(f'Rolling window: {window} months')
print(f'Number of rolling estimates: {len(S_usd) - window}')

In [ ]:
# ─── Rolling Window Estimation ───────────────────────────────────────────────
rolling_usd = {'kappa': [], 'theta': [], 'sigma': [], 'date': []}
rolling_khr = {'kappa': [], 'theta': [], 'sigma': [], 'date': []}

n = len(S_usd)
total_windows = n - window

for i in range(total_windows):
    if i % 20 == 0:
        print(f'  Estimating window {i+1}/{total_windows}...')
    
    window_usd = S_usd[i:i+window]
    window_khr = S_khr[i:i+window]
    
    est_usd = estimate_ou(window_usd, dt)
    est_khr = estimate_ou(window_khr, dt)
    
    rolling_usd['kappa'].append(est_usd['kappa'])
    rolling_usd['theta'].append(est_usd['theta'])
    rolling_usd['sigma'].append(est_usd['sigma'])
    rolling_usd['date'].append(dates[i + window - 1])
    
    rolling_khr['kappa'].append(est_khr['kappa'])
    rolling_khr['theta'].append(est_khr['theta'])
    rolling_khr['sigma'].append(est_khr['sigma'])
    rolling_khr['date'].append(dates[i + window - 1])

# Convert to DataFrames
roll_usd_df = pd.DataFrame(rolling_usd).set_index('date')
roll_khr_df = pd.DataFrame(rolling_khr).set_index('date')

print(f'\nRolling estimation complete: {total_windows} windows estimated.')

In [ ]:
# ─── FIGURE 10: Rolling Parameter Evolution ──────────────────────────────────
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

param_configs = [
    ('theta', 'θ — Long-Run Equilibrium Spread (%)', 'θ'),
    ('kappa', 'κ — Mean Reversion Speed', 'κ'),
    ('sigma', 'σ — Volatility', 'σ'),
]

for ax, (param, title, symbol) in zip(axes, param_configs):
    ax.plot(roll_usd_df.index, roll_usd_df[param], color='#1565C0',
            linewidth=1.3, label=f'{symbol} (USD)', alpha=0.9)
    ax.plot(roll_khr_df.index, roll_khr_df[param], color='#C62828',
            linewidth=1.3, label=f'{symbol} (KHR)', alpha=0.9)
    
    # COVID shading
    ax.axvspan(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-12-31'),
               alpha=0.1, color='grey')
    
    ax.set_title(title, fontweight='bold', fontsize=12)
    ax.set_ylabel(symbol)
    ax.legend(loc='upper right', fontsize=9)
    ax.grid(True, alpha=0.3)

# Event annotations on bottom panel
ax_bottom = axes[2]
events = [
    ('2020-03-01', 'COVID'),
    ('2022-03-01', 'Fed\nHikes'),
    ('2023-07-01', 'Fed\nPeak'),
    ('2024-09-01', 'Fed\nCuts'),
]
for date, label in events:
    for ax in axes:
        ax.axvline(x=pd.Timestamp(date), color='grey', linestyle=':', alpha=0.4, linewidth=0.8)

ax_bottom.set_xlabel('Date')
ax_bottom.xaxis.set_major_locator(mdates.YearLocator())
ax_bottom.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)

fig.suptitle('Figure 10: Rolling 24-Month OU Parameter Evolution',
             fontweight='bold', fontsize=14, y=1.01)
plt.tight_layout()
plt.savefig('../figures/fig10_rolling_parameters.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: fig10_rolling_parameters.png')

In [ ]:
# ─── Convergence / Divergence Analysis ───────────────────────────────────────
# Compute rolling ratio of KHR/USD parameters
theta_ratio = roll_khr_df['theta'] / roll_usd_df['theta']
sigma_ratio = roll_khr_df['sigma'] / roll_usd_df['sigma']

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 7), sharex=True)

ax1.plot(theta_ratio.index, theta_ratio, color='#4A148C', linewidth=1.3)
ax1.axhline(y=1, color='grey', linestyle='--', alpha=0.5)
ax1.axvspan(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-12-31'), alpha=0.1, color='grey')
ax1.set_ylabel('θ_KHR / θ_USD')
ax1.set_title('Rolling Ratio: θ_KHR / θ_USD', fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2.plot(sigma_ratio.index, sigma_ratio, color='#E65100', linewidth=1.3)
ax2.axhline(y=1, color='grey', linestyle='--', alpha=0.5)
ax2.axvspan(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-12-31'), alpha=0.1, color='grey')
ax2.set_xlabel('Date')
ax2.set_ylabel('σ_KHR / σ_USD')
ax2.set_title('Rolling Ratio: σ_KHR / σ_USD', fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.xaxis.set_major_locator(mdates.YearLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('../figures/fig10b_convergence.png', dpi=300, bbox_inches='tight')
plt.show()

print(f'\nθ ratio trend: start = {theta_ratio.iloc[0]:.2f}, end = {theta_ratio.iloc[-1]:.2f}')
print(f'σ ratio trend: start = {sigma_ratio.iloc[0]:.2f}, end = {sigma_ratio.iloc[-1]:.2f}')
if theta_ratio.iloc[-1] < theta_ratio.iloc[0]:
    print('→ USD and KHR equilibrium spreads are CONVERGING over time.')
else:
    print('→ USD and KHR equilibrium spreads are DIVERGING over time.')

In [ ]:
# ─── Save Rolling Results ────────────────────────────────────────────────────
rolling_export = pd.DataFrame({
    'date': roll_usd_df.index,
    'kappa_usd': roll_usd_df['kappa'].values,
    'theta_usd': roll_usd_df['theta'].values,
    'sigma_usd': roll_usd_df['sigma'].values,
    'kappa_khr': roll_khr_df['kappa'].values,
    'theta_khr': roll_khr_df['theta'].values,
    'sigma_khr': roll_khr_df['sigma'].values,
})
rolling_export.to_csv('../data/processed/rolling_ou_parameters.csv', index=False)
print('Saved: rolling_ou_parameters.csv')

---
## Summary

The rolling window analysis reveals how credit risk dynamics evolve over time. Key insights:

1. **Parameter stability**: Are the OU parameters stable or do they shift significantly?
2. **COVID impact**: How did parameters react during 2020–2021?
3. **Convergence**: Are USD and KHR risk dynamics becoming more similar over time, consistent with de-dollarization progress?
4. **Rolling results saved to `rolling_ou_parameters.csv`** for reference.